<a href="https://colab.research.google.com/github/MorningStarTM/word-ordering-/blob/main/word_ordering_using_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing Dataset

In [54]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [55]:
#read the data
dt = pd.read_csv("/content/word_df.csv")

In [56]:
or_cor = []
un_cor = []
for i in range(len(dt['Ordered'])):
  text = dt['Ordered'][i].replace(".","")
  if len(text.split()) == 3:
    or_cor.append(text)
  text = dt['Unordered'][i]
  if len(text.split()) == 3:
    un_cor.append(text)

In [57]:
df = pd.DataFrame()

In [58]:
df['Ordered'] = or_cor
df['Unordered'] = un_cor

In [59]:
df.head()

,Ordered,Unordered
0,I love dogs,I dogs love
1,She plays tennis,She plays tennis
2,They drink coffee,drink coffee They
3,He eats pizza,pizza He eats
4,We go shopping,shopping We go


In [60]:
#list for words and text
input_words = []
input_text = []
target_words = []
target_text = []

In [61]:
#split the unordered text and append into list
for i in range(len(df)):
    data = df['Unordered'][i]
    data = data.lower()
    word_list = data.split(' ')
    if len(word_list) <= 3:
      input_text.append(word_list)
    for word in word_list:
      if word not in input_words:
        input_words.append(word)

In [62]:
#split the ordered text and append into list
for i in range(len(df)):
    data = df['Ordered'][i]
    data = data.lower()
    word_list = data.split(' ')
    if len(word_list) <= 3:
      target_text.append(word_list)
    for word in word_list:
      if word not in target_words:
        target_words.append(word)

In [63]:
#sort the list
input_words = sorted(list(input_words))

In [64]:
len(input_words)

337

In [65]:
vocabulary = len(input_words)

In [66]:
#assign the index for each word
input_word_index = dict(
    (char, i) for i, char in enumerate(input_words)
)

In [67]:
#create the vector for sentence
encoder_input_data = np.zeros(
    (len(input_text), 3, 337), dtype='float32'
)

decoder_input_data = np.zeros(
    (len(input_text), 3, 337), dtype='float32'
)

decoder_target_data = np.zeros(
    (len(input_text), 3, 337), dtype='float32'
)

In [68]:
decoder_input_data.shape

(1038, 3, 337)

In [69]:
#preparing vector 
for j in range(len(input_text)):
  for i in range(len(input_text[j])):
    value = input_word_index[input_text[j][i]]
    encoder_input_data[j][i][value] = 1

In [70]:
#preparing vector 
for j in range(len(target_text)):
  for i in range(len(target_text[j])):
    value = input_word_index[target_text[j][i]]
    decoder_input_data[j][i][value] = 1

In [71]:
input_word_index.items()

dict_items([('abroad', 0), ('animations', 1), ('anthropology', 2), ('appliances', 3), ('archaeology', 4), ('archery', 5), ('art', 6), ('articles', 7), ('astronomy', 8), ('attend', 9), ('attends', 10), ('attentively', 11), ('bake', 12), ('bakes', 13), ('ballet', 14), ('baseball', 15), ('basketball', 16), ('beautifully', 17), ('bicycles', 18), ('bike', 19), ('bikes', 20), ('biking', 21), ('biology', 22), ('birds', 23), ('birdwatching', 24), ('birthdays', 25), ('blogs', 26), ('boardgames', 27), ('books', 28), ('botany', 29), ('boxing', 30), ('breakfast', 31), ('breaks', 32), ('build', 33), ('buildings', 34), ('builds', 35), ('business', 36), ('cakes', 37), ('calligraphy', 38), ('camping', 39), ('canoeing', 40), ('cards', 41), ('care', 42), ('carefully', 43), ('cars', 44), ('cartoons', 45), ('cats', 46), ('celebrate', 47), ('chemistry', 48), ('chess', 49), ('chocolate', 50), ('chores', 51), ('church', 52), ('cities', 53), ('classes', 54), ('clean', 55), ('climb', 56), ('clothes', 57), ('cl

In [72]:
decoder_target_data = decoder_input_data.copy()

In [73]:
decoder_target_data[0].shape

(3, 337)

In [74]:
temp = []

In [76]:
#teacher forcing
for i in range(len(decoder_target_data)):
  a = np.insert(decoder_target_data[i], 0, np.zeros(337))
  a = a[:-337]
  a = a.reshape(3, 337)
  temp.append(a)  

In [77]:
temp = np.array(temp)

In [82]:
#[loves she you] -> [she loves you] === [2 1 3] -> [1 2 3]  but decoder takes [0 1 2]
temp[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)